In [1]:
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 


import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium 

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.19.0               |             py_0          53 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.19.0-py_0       conda-forge

The following packages will be UPDATED:

   

In [3]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_5abaf34aa1d04ed2b8cd96f55afddb69 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='F6Ou8LY5piNkPkbO_MQf04PyqlsYvmiPTDrCZYJ6CzWf',
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_5abaf34aa1d04ed2b8cd96f55afddb69.get_object(Bucket='datasciencecapstone-donotdelete-pr-64lgboidvcwnwa',Key='Toronto Data_geo.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_Toronto= pd.read_csv(body)
df_Toronto.head(5)



,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [4]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [8]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)


for lat, lng, borough, neighbourhood in zip(df_Toronto['Latitude'], df_Toronto['Longitude'], df_Toronto['Borough'], df_Toronto['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

*Visualizing all Toronto Neighbourhoods in order to define next groupings

In [121]:
York_data = df_Toronto[df_Toronto['Borough'] == 'East York'].reset_index(drop=True)
York_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
1,M4C,East York,Woodbine Heights,43.695344,-79.318389
2,M4G,East York,Leaside,43.709060,-79.363452
3,M4H,East York,Thorncliffe Park,43.705369,-79.349372
4,M4J,East York,East Toronto,43.685347,-79.338106


*All Neighborhoud Data will be based on the East York Borough

In [122]:
address = 'East York, Toronto'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of East York, Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of East York, Toronto are 43.6913391, -79.3278212.


In [123]:
map_York = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(York_data['Latitude'], York_data['Longitude'], York_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_York)  
    
map_York

Visualizing the East York Neighborhoods prior to clustering

In [ ]:
CLIENT_ID 
CLIENT_SECRET 
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


In [125]:
York_data.loc[0, 'Neighbourhood']

'Parkview Hill, Woodbine Gardens'

In [126]:
neighbourhood_latitude = York_data.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = York_data.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = York_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Parkview Hill, Woodbine Gardens are 43.7063972, -79.309937.


In [127]:
LIMIT = 100 # limit of number of venues returned by Foursquare API



radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=FW3I2WWKTVVRBVCHKYKN4QEU1SGSOZAEV2KPJMPCN5CO0QAF&client_secret=PCREM2YUL53YDCQWFMDJ412M3NIJQANYLN4CWZPZVE52UL4J&v=20180605&ll=43.7063972,-79.309937&radius=500&limit=100'

In [128]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ccf2b6c1ed2196c3fab1c14'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4b5a3842f964a52023b528e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/gastropub_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d155941735',
         'name': 'Gastropub',
         'pluralName': 'Gastropubs',
         'primary': True,
         'shortName': 'Gastropub'}],
       'id': '4b5a3842f964a52023b528e3',
       'location': {'address': "804 O'Connor Dr",
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'St Clair E',
        'distance': 249,
        'formattedAddress': ["804 O'Connor Dr (St Clair E)",
         'Toronto ON M4B 2S9',
         'Canada'],
        'labeledLatLngs': [{'label

In [130]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [131]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Jawny Bakers,Gastropub,43.705783,-79.312913
1,Toronto Climbing Academy,Rock Climbing Spot,43.709136,-79.314790
2,East York Gymnastics,Gym / Fitness Center,43.710654,-79.309279
3,Shoppers Drug Mart,Pharmacy,43.705892,-79.312410
4,TD Canada Trust,Bank,43.705740,-79.312270


In [132]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [133]:
York_venues = getNearbyVenues(names=York_data['Neighbourhood'],
                                   latitudes=York_data['Latitude'],
                                   longitudes=York_data['Longitude']
                                  )

Parkview Hill, Woodbine Gardens
Woodbine Heights
Leaside
Thorncliffe Park
East Toronto


In [134]:
print(York_venues.shape)
York_venues.head()

(74, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,Jawny Bakers,43.705783,-79.312913,Gastropub
1,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,Toronto Climbing Academy,43.709136,-79.314790,Rock Climbing Spot
2,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,East York Gymnastics,43.710654,-79.309279,Gym / Fitness Center
3,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,Shoppers Drug Mart,43.705892,-79.312410,Pharmacy
4,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,TD Canada Trust,43.705740,-79.312270,Bank


In [135]:
# one hot encoding
York_onehot = pd.get_dummies(York_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
York_onehot['Neighborhood'] = York_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [York_onehot.columns[-1]] + list(York_onehot.columns[:-1])
York_onehot = York_onehot[fixed_columns]

York_onehot.head()

,Neighborhood,Asian Restaurant,Athletics & Sports,Bagel Shop,Bank,Beer Store,Bike Shop,Breakfast Spot,Brewery,Burger Joint,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Curling Ice,Dessert Shop,Electronics Store,Fast Food Restaurant,Fish & Chips Shop,Furniture / Home Store,Gastropub,Grocery Store,Gym,Gym / Fitness Center,Housing Development,Indian Restaurant,Intersection,Liquor Store,Metro Station,Mexican Restaurant,Park,Pet Store,Pharmacy,Pizza Place,Restaurant,Rock Climbing Spot,Sandwich Place,Shopping Mall,Skating Rink,Smoothie Shop,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Warehouse Store,Yoga Studio
0,"Parkview Hill, Woodbine Gardens",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Parkview Hill, Woodbine Gardens",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,"Parkview Hill, Woodbine Gardens",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Parkview Hill, Woodbine Gardens",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Parkview Hill, Woodbine Gardens",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [136]:
York_grouped = York_onehot.groupby('Neighborhood').mean().reset_index()
York_grouped

,Neighborhood,Asian Restaurant,Athletics & Sports,Bagel Shop,Bank,Beer Store,Bike Shop,Breakfast Spot,Brewery,Burger Joint,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Curling Ice,Dessert Shop,Electronics Store,Fast Food Restaurant,Fish & Chips Shop,Furniture / Home Store,Gastropub,Grocery Store,Gym,Gym / Fitness Center,Housing Development,Indian Restaurant,Intersection,Liquor Store,Metro Station,Mexican Restaurant,Park,Pet Store,Pharmacy,Pizza Place,Restaurant,Rock Climbing Spot,Sandwich Place,Shopping Mall,Skating Rink,Smoothie Shop,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Warehouse Store,Yoga Studio
0,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Leaside,0.000000,0.000000,0.030303,0.030303,0.030303,0.030303,0.030303,0.030303,0.060606,0.030303,0.090909,0.00,0.000000,0.000000,0.030303,0.030303,0.000000,0.030303,0.060606,0.000000,0.030303,0.030303,0.000000,0.000000,0.000000,0.000000,0.030303,0.00,0.030303,0.000000,0.030303,0.000000,0.030303,0.030303,0.000000,0.030303,0.030303,0.000000,0.030303,0.090909,0.030303,0.030303,0.030303,0.000000,0.000000
2,"Parkview Hill, Woodbine Gardens",0.000000,0.076923,0.000000,0.076923,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.153846,0.000000,0.000000,0.076923,0.000000,0.000000,0.076923,0.000000,0.000000,0.076923,0.000000,0.00,0.000000,0.000000,0.076923,0.076923,0.153846,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Thorncliffe Park,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.058824,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.058824,0.058824,0.117647,0.000000,0.058824,0.00,0.000000,0.058824,0.000000,0.058824,0.058824,0.000000,0.000000,0.117647,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.058824,0.058824
4,Woodbine Heights,0.142857,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.142857,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.142857,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [137]:
num_top_venues = 5

for hood in York_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = York_grouped[York_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----East Toronto----
                venue  freq
0                Park  0.50
1   Convenience Store  0.25
2       Metro Station  0.25
3  Rock Climbing Spot  0.00
4        Intersection  0.00


----Leaside----
                    venue  freq
0             Coffee Shop  0.09
1     Sporting Goods Shop  0.09
2            Burger Joint  0.06
3  Furniture / Home Store  0.06
4       Electronics Store  0.03


----Parkview Hill, Woodbine Gardens----
                  venue  freq
0           Pizza Place  0.15
1  Fast Food Restaurant  0.15
2  Gym / Fitness Center  0.08
3                  Bank  0.08
4    Athletics & Sports  0.08


----Thorncliffe Park----
                  venue  freq
0        Sandwich Place  0.12
1     Indian Restaurant  0.12
2  Gym / Fitness Center  0.06
3         Grocery Store  0.06
4       Warehouse Store  0.06


----Woodbine Heights----
              venue  freq
0  Asian Restaurant  0.14
1      Skating Rink  0.14
2              Park  0.14
3          Pharmacy  0.14
4       Curling

In [138]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [139]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = York_grouped['Neighborhood']

for ind in np.arange(York_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(York_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,Park,Convenience Store,Metro Station,Yoga Studio,Furniture / Home Store,Fish & Chips Shop,Fast Food Restaurant,Electronics Store,Dessert Shop,Curling Ice
1,Leaside,Sporting Goods Shop,Coffee Shop,Furniture / Home Store,Burger Joint,Beer Store,Mexican Restaurant,Electronics Store,Fish & Chips Shop,Grocery Store,Gym
2,"Parkview Hill, Woodbine Gardens",Pizza Place,Fast Food Restaurant,Rock Climbing Spot,Intersection,Pet Store,Pharmacy,Gastropub,Gym / Fitness Center,Breakfast Spot,Athletics & Sports
3,Thorncliffe Park,Indian Restaurant,Sandwich Place,Yoga Studio,Pharmacy,Bank,Burger Joint,Coffee Shop,Grocery Store,Warehouse Store,Housing Development
4,Woodbine Heights,Asian Restaurant,Pharmacy,Beer Store,Skating Rink,Curling Ice,Cosmetics Shop,Park,Furniture / Home Store,Fish & Chips Shop,Fast Food Restaurant


In [140]:
# set number of clusters
kclusters = 5

York_grouped_clustering = York_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(York_grouped_clustering)

# check cluster labels generated for each row in the dataframe
# add clustering labels
kmeans.labels_[0:10] 

array([2, 3, 4, 1, 0], dtype=int32)

In [143]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

York_merged = York_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
York_merged = York_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

York_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,4,Pizza Place,Fast Food Restaurant,Rock Climbing Spot,Intersection,Pet Store,Pharmacy,Gastropub,Gym / Fitness Center,Breakfast Spot,Athletics & Sports
1,M4C,East York,Woodbine Heights,43.695344,-79.318389,0,Asian Restaurant,Pharmacy,Beer Store,Skating Rink,Curling Ice,Cosmetics Shop,Park,Furniture / Home Store,Fish & Chips Shop,Fast Food Restaurant
2,M4G,East York,Leaside,43.709060,-79.363452,3,Sporting Goods Shop,Coffee Shop,Furniture / Home Store,Burger Joint,Beer Store,Mexican Restaurant,Electronics Store,Fish & Chips Shop,Grocery Store,Gym
3,M4H,East York,Thorncliffe Park,43.705369,-79.349372,1,Indian Restaurant,Sandwich Place,Yoga Studio,Pharmacy,Bank,Burger Joint,Coffee Shop,Grocery Store,Warehouse Store,Housing Development
4,M4J,East York,East Toronto,43.685347,-79.338106,2,Park,Convenience Store,Metro Station,Yoga Studio,Furniture / Home Store,Fish & Chips Shop,Fast Food Restaurant,Electronics Store,Dessert Shop,Curling Ice


In [144]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(York_merged['Latitude'], York_merged['Longitude'], York_merged['Neighbourhood'], York_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [146]:
York_merged.loc[York_merged['Cluster Labels'] == 0, York_merged.columns[[2] + list(range(5, York_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Woodbine Heights,0,Asian Restaurant,Pharmacy,Beer Store,Skating Rink,Curling Ice,Cosmetics Shop,Park,Furniture / Home Store,Fish & Chips Shop,Fast Food Restaurant


In [147]:
York_merged.loc[York_merged['Cluster Labels'] == 1, York_merged.columns[[2] + list(range(5, York_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Thorncliffe Park,1,Indian Restaurant,Sandwich Place,Yoga Studio,Pharmacy,Bank,Burger Joint,Coffee Shop,Grocery Store,Warehouse Store,Housing Development


In [148]:
York_merged.loc[York_merged['Cluster Labels'] == 2, York_merged.columns[[2] + list(range(5, York_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East Toronto,2,Park,Convenience Store,Metro Station,Yoga Studio,Furniture / Home Store,Fish & Chips Shop,Fast Food Restaurant,Electronics Store,Dessert Shop,Curling Ice


In [149]:
York_merged.loc[York_merged['Cluster Labels'] == 3, York_merged.columns[[2] + list(range(5, York_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Leaside,3,Sporting Goods Shop,Coffee Shop,Furniture / Home Store,Burger Joint,Beer Store,Mexican Restaurant,Electronics Store,Fish & Chips Shop,Grocery Store,Gym


In [150]:
York_merged.loc[York_merged['Cluster Labels'] == 4, York_merged.columns[[2] + list(range(5, York_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Parkview Hill, Woodbine Gardens",4,Pizza Place,Fast Food Restaurant,Rock Climbing Spot,Intersection,Pet Store,Pharmacy,Gastropub,Gym / Fitness Center,Breakfast Spot,Athletics & Sports


The above breaks out the individual neighborhoods by their venue clustering. 
3 out of 5 clusters have a restaurant as the 1st most common venue. The other two are a park and sports goods shop, which are closesly related by subject. 